In [62]:
import polars as pl
import duckdb
import altair as alt
# import plotly.express as px

In [49]:
color_scale = [
    "#007F77",  # teal-700  – accent, starts the scale
    "#4385BE",  # blue-600
    "#8B7EC8",  # purple-600
    "#D83232",  # red-700
    "#BC6F00",  # orange-700
    "#6F6E69",  # gray-600  – neutral end-stop
]

In [30]:
con = duckdb.connect('../data/03_gold/monitor_campista_pharma_ads_1880_1884.duckdb', True)

In [10]:
con.sql("SHOW ALL TABLES;").pl()

database,schema,name,column_names,column_types,temporary
str,str,str,list[str],list[str],bool
"""monitor_campista_pharma_ads_18…","""main""","""alinhamento""","[""Identificador"", ""alinhamento""]","[""VARCHAR"", ""VARCHAR""]",false
"""monitor_campista_pharma_ads_18…","""main""","""anuncios""","[""Identificador"", ""ID"", … ""image_url""]","[""VARCHAR"", ""BIGINT"", … ""VARCHAR""]",false
"""monitor_campista_pharma_ads_18…","""main""","""autorizacoes""","[""Identificador"", ""autorizacoes""]","[""VARCHAR"", ""VARCHAR""]",false
"""monitor_campista_pharma_ads_18…","""main""","""detalhamento_do_efeito""","[""Identificador"", ""detalhamento_do_efeito""]","[""VARCHAR"", ""VARCHAR""]",false
"""monitor_campista_pharma_ads_18…","""main""","""detalhamento_forma_de_uso""","[""Identificador"", ""detalhamento_forma_de_uso""]","[""VARCHAR"", ""VARCHAR""]",false
…,…,…,…,…,…
"""monitor_campista_pharma_ads_18…","""main""","""tipificacao_da_imagem_aprox""","[""Identificador"", ""tipificacao_da_imagem_aprox""]","[""VARCHAR"", ""VARCHAR""]",false
"""monitor_campista_pharma_ads_18…","""main""","""tipo_de_produto""","[""Identificador"", ""tipo_de_produto""]","[""VARCHAR"", ""VARCHAR""]",false
"""monitor_campista_pharma_ads_18…","""main""","""variacao_tipografica""","[""Identificador"", ""variacao_tipografica""]","[""VARCHAR"", ""VARCHAR""]",false


In [19]:
con.query("""
select 
    *
from veiculacoes
""").pl()

Identificador,Ano,Edição,Página,Coluna(s) ocupadas,Número de Colunas,Orientação,ano_edicao
str,i64,i64,i64,str,i64,str,str
"""tonico_oriental_cabello""",1880,2,4,"""1""",1,"""↥""","""1880_002"""
"""pilulas_vegeates_assucaradas_b…",1880,6,4,"""5""",1,"""↥""","""1880_006"""
"""novo_depurativo_extracto_salsa…",1880,6,3,"""4,5""",2,"""↥""","""1880_006"""
"""oleo_figado_bacalhau""",1880,8,4,"""5""",1,"""↥""","""1880_008"""
"""novo_depurativo_extracto_salsa…",1880,8,4,"""3,4""",2,"""↥""","""1880_008"""
…,…,…,…,…,…,…,…
"""cura_gonorrhea_sem_massantes_i…",1884,306,4,"""3""",1,"""↥""","""1884_306"""
"""oleo_ducoux_comprido""",1884,306,4,"""3""",1,"""↥""","""1884_306"""
"""importante_descoberta_xarope_s…",1884,306,4,"""3,4""",2,"""↥""","""1884_306"""


In [31]:
df_ads_by_edition = con.query("""
select 
    ano_edicao,
    count(*) as anuncios,
    min(Página) as menor_pagina,
    max(Página) as maior_pagina,
from veiculacoes
group by 
    ano_edicao
order by 
    ano_edicao
""").pl()

In [32]:
df_ads_by_edition

ano_edicao,anuncios,menor_pagina,maior_pagina
str,i64,i64,i64
"""1880_002""",1,4,4
"""1880_006""",2,3,4
"""1880_008""",2,4,4
"""1880_010""",1,4,4
"""1880_012""",2,4,4
…,…,…,…
"""1884_298""",7,3,4
"""1884_300""",9,3,4
"""1884_302""",8,3,4


In [38]:
df_ads_by_page = con.sql("""
    select
        "Página",
        count(*) as anuncios
    from veiculacoes
    group by
        "Página"
    order by
        Página
    """).pl()

In [57]:
# range_chart = (
#     alt.Chart(df_ads_by_edition)
#     .mark_area()
#     .encode(
#         x=alt.X("ano_edicao:O", title="Edition Year"),
#         y=alt.Y("pagina_primeiro_anuncio:Q", title="Page Number"),
#         y1="pagina_ultimo_anuncio:Q",
#         color=alt.Color("ano:N").scale(range=color_scale),
#     )
# )

# Add line for the first ad page
line_first = (
    alt.Chart(df_ads_by_edition)
    .mark_circle(color="darkblue")
    .encode(x="ano_edicao:O", y="pagina_primeiro_anuncio:Q")
)

# Add line for the last ad page
line_last = (
    alt.Chart(df_ads_by_edition)
    .mark_circle(color=color_scale[-1])
    .encode(x="ano_edicao:O", y="pagina_ultimo_anuncio:Q")
)

# Combine all
chart = line_first + line_last

In [64]:
(
    alt.Chart(df_ads_by_page)
    .mark_bar(size=50)
    .encode(
        x=alt.X("Página:O")
        .title("Edição")
        .axis(labelAngle=0)
        .scale(domain=range(1, 9)),
        y=alt.Y("anuncios").title("Contagem Anúncios"),
    )
)

alt.Chart(...)

In [69]:
ad_edition_page = con.query("""
select 
    Ano,ano_edicao,Página
from veiculacoes
""").pl()

In [113]:
# something wrong with this graph
agg = ad_edition_page.group_by(["Página","Ano"]).len()
alt.Chart(agg).mark_bar().encode(
    x=alt.X('Página:O', title='Página'),
    y=alt.Y('count():Q', stack='zero', title='Páginas'),
    color='Ano:N'
)

alt.Chart(...)

In [120]:
(alt.Chart(ad_edition_page).mark_bar().encode(
    x=alt.X('Página:O', title='Página'),
    y=alt.Y('count():Q', stack='zero', title='Veiculações'),
    color='Ano:N'
).transform_aggregate(
    count='count()',
    groupby=['Ano', 'Página']
).encode(
    x='Página:N',
    y=alt.Y('sum(count):Q', title="Veiculações"),
    color=alt.Color('Ano:N').scale(range=color_scale)
))

alt.Chart(...)

In [126]:
ads_per_edition = (
    alt.Chart(df_ads_by_page)
    .mark_bar(size=70)
    .encode(
        x=alt.X("Página:O")
        .title("Edição")
        .axis(labelAngle=0)
        .scale(domain=range(1, 9)),
        y=alt.Y("anuncios")
        .title("Contagem Anúncios")
        .axis(format=",.0f"),  # Basic formatting without thousands separator
        color=alt.value(color_scale[0]),  # Apply a single color to all bars
    )
)
ads_per_edition

alt.Chart(...)

In [28]:
con.close()